<a href="https://colab.research.google.com/github/Tstrebe2/predicting-text-difficulty/blob/dave-updates/code/dave_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('drive/MyDrive/696/text_difficulty_features.csv',sep='\t')
df.head(5)

,original_text,lemmatized_text,d_chall_score,aoa_mean,aoa_min,aoa_max,conc_rating_mean,conc_rating_min,conc_rating_max,num_lemmas,label
0,There is manuscript evidence that Austen conti...,There be manuscript evidence that Austen conti...,8.533221,5.809310,3.57,12.12,2.495517,1.33,4.57,38.0,1
1,"In a remarkable comparative analysis , Mandaea...","In a remarkable comparative analysis , Mandaea...",12.320171,7.402308,2.89,11.94,2.334286,1.46,4.93,21.0,1
2,"Before Persephone was released to Hermes , who...","Before Persephone be release to Hermes , who h...",5.931500,5.231351,2.78,11.17,2.556486,1.43,4.86,40.0,1
3,Cogeneration plants are commonly found in dist...,Cogeneration plant be commonly find in distric...,7.015012,6.742000,3.56,11.53,3.369655,1.52,4.93,32.0,1
4,Geneva is the second-most-populous city in Swi...,Geneva be the second-most-populous city in Swi...,7.550745,5.455000,3.69,12.62,2.399333,1.43,4.79,22.0,1


In [4]:
df.dtypes

original_text        object
lemmatized_text      object
d_chall_score       float64
aoa_mean            float64
aoa_min             float64
aoa_max             float64
conc_rating_mean    float64
conc_rating_min     float64
conc_rating_max     float64
num_lemmas          float64
label                 int64
dtype: object

In [5]:
# Extract only the columns we'll use for the classifier
df = df[df.columns[:]]

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [7]:
#Commented out so that the Training and Testing sets are preserved

# X = df[df.columns[:-1]]
# y = df.label

# X_train, X_test, y_train, y_test = train_test_split(X,y,
#                                                     test_size=.1,
#                                                     stratify=y,
#                                                     random_state=99)
#  Training_Set = pd.concat([X_train, pd.DataFrame(y_train)], axis=1)
#  Testing_Set = pd.concat([X_test, pd.DataFrame(y_test)], axis=1)
#  Testing_Set.to_csv('drive/Shareddrives/Milestone 2/Testing_set.csv', sep='\t', index=False)
# #  Training_Set.to_csv('drive/Shareddrives/Milestone 2/Training_set.csv', sep='\t', index=False)

In [8]:
train = pd.read_csv('drive/Shareddrives/Milestone 2/Training_set.csv', 
                      sep='\t',index_col=0)

test = pd.read_csv('drive/Shareddrives/Milestone 2/Testing_set.csv', sep='\t',
                     index_col=0)

In [9]:
print(f"Training set dimensions: {train.shape}")
print(f"label counts from training split (order is positive, negative): {train.label.value_counts().values}")

print(f"Testing set dimensions: {test.shape}")
print(f"label counts from training split (order is positive, negative): {test.label.value_counts().values}")


Training set dimensions: (355652, 10)
label counts from training split (order is positive, negative): [184549 171103]
Testing set dimensions: (39517, 10)
label counts from training split (order is positive, negative): [20506 19011]


In [10]:
train.head()

,lemmatized_text,d_chall_score,aoa_mean,aoa_min,aoa_max,conc_rating_mean,conc_rating_min,conc_rating_max,num_lemmas,label
91224,For instance the number of k-permutations of n...,4.542700,5.140000,3.94,9.57,2.577500,1.43,4.55,12.0,1
117728,Singer and bass player Roger Waters gradually ...,7.556000,5.835000,3.69,11.11,2.637727,1.33,4.62,25.0,1
316017,Other page 2005 Macquarie Fields riot 2005 Cro...,10.714000,6.456250,3.89,10.00,3.042500,1.43,4.90,25.0,0
7423,Saint-Denis be founded by Ã tienne RÃ gnault i...,8.887000,5.613637,3.69,9.39,2.545454,1.43,4.96,20.0,1
201052,Pougny be a commune in the Ain department in e...,7.722873,6.232500,2.89,12.05,2.505000,1.43,3.30,11.0,1


In [329]:
X_train = train[train.columns[:-1]]
y_train = train[train.columns[-1]]

X_test = test[test.columns[:-1]]
y_test = test[test.columns[-1]]

In [236]:
metrics_df = {"model_name":[],
              "model_instance":[],
              "train_accuracy": [],
              "train_precision": [],
              "train_recall": [],
              "train_aucprc":[],
              "train_aucroc": [],
              "test_accuracy":[],
              "test_precision":[],
              "test_recall":[],
              "test_aucprc":[],
              "test_aucroc": []
              }

In [326]:
from sklearn. metrics import precision_score, recall_score, precision_recall_curve, roc_auc_score, auc

In [238]:
from sklearn.metrics._plot.precision_recall_curve import plot_precision_recall_curve


In [327]:
def generate_model_name(model):
  for key,value in model.best_params_.items():
    model_name += f" {key}_{value}"
  return model_name

def compute_metrics(model,model_name,X_train_data,y_train_data, X_test_data, y_test_data):
    # extract the model's hyperparameters and make that part of the identifier for the model 

    metrics_df['model_name'].append(model_name)
    metrics_df['model_instance'].append(model)
   
    # run train data
    y_pred_train = model.predict(X_train_data)
    acc = model.score(X_train_data, y_train)
    recall = recall_score(y_train_data,y_pred_train)
    precision = precision_score(y_train_data, y_pred_train)
    y_proba_train = model.predict_proba(X_train_data)[:,1]
    prc = auc(recall,precision)
    roc = roc_auc = roc_auc_score(y_train_data, y_proba_train)
   
   
    metrics_df['train_accuracy'].append(acc)
    metrics_df['train_recall'].append(recall)
    metrics_df['train_precision'].append(precision)
    metrics_df['train_aucprc'].append(prc)
    metrics_df['train_aucroc'].append(roc)


    #run test data
    y_pred = model.predict(X_test_data)
    acc = model.score(X_test_data, y_test)
    recall = recall_score(y_test,y_pred)
    precision = precision_score(y_test, y_pred)
    y_proba = model.predict_proba(X_test_data)[:,1]
    prc = auc(recall,precision)
    roc = roc_auc = roc_auc_score(y_test, y_proba)

    metrics_df['test_accuracy'].append(acc)
    metrics_df['test_recall'].append(recall)
    metrics_df['test_precision'].append(precision)
    metrics_df['test_aucprc'].append(prc)
    metrics_df['test_aucroc'].append(roc)








### **Naive Bayes with Tfidf Vectorizer**

In [242]:
# NaiveBayes Classifier with Tfidf Vectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

In [243]:
# due to the amount of time it took to run the tfidf vectorizer, I manually 
# tested min_df settings and determined the optimal range to be in the 20s
# I also ran manual models on ngram ranges and saw better performance on 
# bigrams compared to trigrams


pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', BernoulliNB())
                    ])

parameters = {
    'tfidf__min_df': [25,30],
    'tfidf__stop_words': ['english'],
    'tfidf__ngram_range': [(1, 2)],
    'clf__alpha': [0.001,0.01,0.1,1,5,10,50, 100]
}

In [244]:
grid_bnb_f1 = GridSearchCV(pipeline, 
                            param_grid=parameters,
                            scoring='f1', 
                            cv=3, 
                            n_jobs=-1).fit(X_train['lemmatized_text'],y_train)
grid_bnb_f1.best_params_

{'clf__alpha': 0.001,
 'tfidf__min_df': 30,
 'tfidf__ngram_range': (1, 2),
 'tfidf__stop_words': 'english'}

In [245]:
vec = grid_bnb_f1.best_estimator_.named_steps['tfidf']
best_model = grid_bnb_acc.best_estimator_.named_steps['clf']
neg_class = best_model.feature_log_prob_[0, :].argsort()[::-1]
pos_class = best_model.feature_log_prob_[1, :].argsort()[::-1]

In [246]:
print("Top words in negative class:", np.take(vec.get_feature_names_out(), neg_class[:10]))
print("words in positive class:", np.take(vec.get_feature_names_out(), pos_class[:10]))

Top words in negative class: ['bear' 'use' 'make' 'france' 'city' 'region' 'united' 'department'
 'north' 'know']
words in positive class: ['bear' 'use' 'france' 'know' 'city' 'department' 'commune' 'united'
 'year' 'include']


In [52]:
grid_bnb_f1.best_estimator_.score(X_test['lemmatized_text'], y_test)

0.6653086013614394

In [ ]:
# tf_vec = TfidfVectorizer(min_df = 30,
#                          stop_words='english',
#                          ngram_range=(1, 2))

In [ ]:
# X_train_vec = tf_vec.fit_transform(X_train['lemmatized_text'])
# X_test_vec = tf_vec.transform(X_test['lemmatized_text'])

In [32]:
# bnb = BernoulliNB(alpha=.1).fit(X_train_vec,y_train)
# bnb.score(X_test_vec,y_test)
# manual_params = {'alpha': [0.001, 0.01, 0.1, 1,5,10,50, 100]}
# grid_bn_acc = GridSearchCV(BernoulliNB(), param_grid=param_grid)
# grid_bn_acc.fit(X_train_vec,y_train)
# grid_bn_acc.best_estimator_.score(tf_vec.transform(X_test['lemmatized_text']),y_test)

In [330]:
model_name  = 'Tf-idf BNB alpha_.001 min_df_30 ngram_1_2'

# X_test_vec = tf_vec.transform(X_test['lemmatized_text'])
compute_metrics(grid_bnb_f1.best_estimator_,
                model_name,
                X_train['lemmatized_text'],
                y_train,
                X_test['lemmatized_text'],
                y_test)



TypeError: ignored

# Mads Features

In [249]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [250]:
# X_train_mads_features = X_train.iloc[:,1:9]
# mads_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
# X_train_mads_features_imp = mads_mean.fit_transform(X_train_mads_features)

In [262]:
cols = ['d_chall_score','aoa_mean','aoa_min','aoa_max','conc_rating_mean','conc_rating_min', 'conc_rating_max','num_lemmas']
X_train_mads_features = X_train.iloc[:][cols]
X_test_mads_features = X_test.iloc[:][cols]
X_train_mads_features.describe()

,d_chall_score,aoa_mean,aoa_min,aoa_max,conc_rating_mean,conc_rating_min,conc_rating_max,num_lemmas
count,355652.000000,355645.000000,355652.000000,355645.000000,352989.000000,352989.000000,352989.000000,355652.000000
mean,7.556820,5.754599,3.395488,10.419940,2.673894,1.472457,4.460836,19.759959
std,2.687109,1.004769,0.823235,2.596776,0.355685,0.321249,0.568358,10.539020
min,0.000000,0.000000,0.000000,0.000000,1.190000,1.040000,1.190000,0.000000
25%,5.729000,5.227647,2.890000,8.780000,2.447143,1.430000,4.180000,12.000000
50%,7.461543,5.676970,3.280000,10.500000,2.639091,1.430000,4.700000,18.000000
75%,9.158327,6.205294,3.690000,12.110000,2.863000,1.440000,4.880000,25.000000
max,18.716400,18.330000,18.330000,25.000000,5.000000,5.000000,5.000000,76.000000


In [266]:
mads_pipeline = Pipeline([('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
                          ('scaler', StandardScaler()),
                          ('clf', BernoulliNB())
                         ])

parameters = {
    'clf__alpha': [0.001,0.01,0.1,1,5,10,50, 100]
}

In [267]:
grid_bnb_mads_f1 = (GridSearchCV(mads_pipeline,
                                  param_grid=parameters,
                                  scoring='f1',
                                  n_jobs=-1, 
                                  cv=3).fit(X_train_mads_features,y_train))

In [268]:
# old code pre-pipeline
# param_grid = {'alpha': [0.001, 0.01,0.1,.5,1,2,3,4,5,10,25,50,100,500]}
# grid_mnb_mads_acc_alt = GridSearchCV(MultinomialNB(), param_grid=param_grid)
# grid_mnb_mads_acc_alt.fit(X_train_mads_features,y_train)

In [269]:
best_model = grid_bnb_mads_f1.best_estimator_.named_steps['clf']


neg_class = best_model.feature_log_prob_[0, :].argsort()[::-1]
pos_class = best_model.feature_log_prob_[1, :].argsort()[::-1]

print(np.take(cols, neg_class[:10]))
print(np.take(cols, pos_class[:10]))





['conc_rating_max' 'aoa_min' 'conc_rating_mean' 'aoa_max' 'd_chall_score'
 'aoa_mean' 'num_lemmas' 'conc_rating_min']
['conc_rating_max' 'aoa_max' 'd_chall_score' 'aoa_mean' 'num_lemmas'
 'aoa_min' 'conc_rating_mean' 'conc_rating_min']


In [270]:
# Negative Class Feature Importance by Log Probabilities
list(zip(np.take(cols, neg_class[:10]),
         [best_model.feature_log_prob_[0,idx] for idx in neg_class]))

[('conc_rating_max', -0.4863358785204266),
 ('aoa_min', -0.671873412065711),
 ('conc_rating_mean', -0.7231359913692863),
 ('aoa_max', -0.8588584768060237),
 ('d_chall_score', -0.8828446602022311),
 ('aoa_mean', -0.9743919150440146),
 ('num_lemmas', -1.0717187433192912),
 ('conc_rating_min', -1.749705504150402)]

In [271]:
# Positive Class Feature Importance by Log Probabilities
list(zip(np.take(cols, pos_class[:10]),
         [best_model.feature_log_prob_[1,idx] for idx in pos_class]))

[('conc_rating_max', -0.41636423046296045),
 ('aoa_max', -0.5133973531697382),
 ('d_chall_score', -0.6057990039610655),
 ('aoa_mean', -0.6225224713905568),
 ('num_lemmas', -0.6535876533784393),
 ('aoa_min', -0.7897565029673448),
 ('conc_rating_mean', -0.8260370586110906),
 ('conc_rating_min', -2.2203843163072783)]

In [272]:
grid_bnb_mads_f1.best_estimator_

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('clf', BernoulliNB(alpha=0.001))])

In [259]:
X_test_mads_features

,d_chall_score,aoa_mean,aoa_min,aoa_max,conc_rating_mean,conc_rating_min,conc_rating_max,num_lemmas
196765,7.590821,5.722857,2.89,11.59,2.312857,1.43,3.84,19.0
145609,7.344964,6.081429,3.69,11.56,2.866923,1.43,4.18,17.0
333830,5.727975,5.956154,3.68,13.33,3.014167,1.43,4.90,16.0
336288,5.795817,6.546957,3.57,14.50,3.496190,1.33,5.00,24.0
247542,7.174366,5.413333,3.63,8.56,2.910000,1.43,4.86,12.0
...,...,...,...,...,...,...,...,...
300022,14.481956,7.105000,3.89,10.58,3.814000,2.97,4.90,9.0
360562,7.308000,5.510000,3.95,9.94,2.370667,1.33,4.90,20.0
139935,7.111650,6.332727,3.06,11.72,2.233043,1.33,4.93,24.0
22927,8.159213,5.425000,2.89,11.14,2.819091,1.43,4.86,31.0


In [273]:
grid_bnb_mads_f1.best_estimator_.score(X_train_mads_features, y_train)

0.6124385635396399

In [274]:
model_name = 'MADS BNB alpha .001'
compute_metrics(grid_bnb_mads_f1.best_estimator_,
                model_name,
                X_train_mads_features,
                y_train,
                X_test_mads_features,
                y_test)

In [275]:
pd.DataFrame(metrics_df)

,model_name,model_instance,train_accuracy,train_precision,train_recall,train_aucprc,train_aucroc,test_accuracy,test_precision,test_recall,test_aucprc,test_aucroc
0,Tf-idf BNB alpha_.001 min_df_30 ngram_1_2,"(TfidfVectorizer(min_df=30, ngram_range=(1, 2)...",0.684894,0.718241,0.646273,"([0.5257042424035254, 0.5258272145244425, 0.52...",0.757069,0.665309,0.697354,0.627231,"([0.5280696332921302, 0.5280574798485745, 0.52...",0.734647
1,MADS BNB alpha .001,"(SimpleImputer(), MinMaxScaler(), BernoulliNB(...",0.520343,0.519756,0.994928,"([0.5189033099771687, 0.5197554278597107, 0.51...",0.501709,0.519852,0.519504,0.994977,"([0.518915909608523, 0.5195039975556348, 1.0, ...",0.501195
2,MADS BNB alpha .001,"(SimpleImputer(), StandardScaler(), BernoulliN...",0.612439,0.623923,0.637186,"([0.5189033099771687, 0.5209164530313577, 0.52...",0.651824,0.608574,0.620319,0.633327,"([0.518915909608523, 0.5208710047115752, 0.522...",0.649156


In [ ]:
# tfidf + Mads features

# tfidf + Mads Features

In [276]:
import scipy
from scipy.sparse import hstack
import numpy as np
from sklearn.compose import ColumnTransformer

In [277]:
# replacing this manual method with the code in the cell below.

# # Create a sparse matrix from the MADS features
# X_sparse = scipy.sparse.coo_matrix(X_train_mads_features)

# # Join both the tfidf training matrix with the sparse MADS features
# X_train_all = scipy.sparse.hstack((X_train_vec, X_sparse)).tocsr()




In [287]:
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import KBinsDiscretizer



# def get_text_list(x):
#   return x.values[:,0]

list_trf = FunctionTransformer(get_text_list)


vectorizer = Pipeline([
              # ('select', list_trf),
              ('tfidf', TfidfVectorizer(min_df=25,
                                        stop_words='english',
                                        ngram_range=(1, 2)))
              ])

mads_pipe = Pipeline([
              ('impute_mean',SimpleImputer(strategy='mean')),
              ('scaler',MinMaxScaler()),
              ('bin',KBinsDiscretizer(n_bins=4))
              ])

preprocessor = ColumnTransformer(transformers=[
    ('mads',mads_pipe,cols),
    ('text', vectorizer, 'lemmatized_text')
    ])


              
pipeline = Pipeline([('preprocessor', preprocessor),
                     ('clf', BernoulliNB())  # classifier
                     ])

In [279]:
X_train

,lemmatized_text,d_chall_score,aoa_mean,aoa_min,aoa_max,conc_rating_mean,conc_rating_min,conc_rating_max,num_lemmas
91224,For instance the number of k-permutations of n...,4.542700,5.140000,3.94,9.57,2.577500,1.43,4.55,12.0
117728,Singer and bass player Roger Waters gradually ...,7.556000,5.835000,3.69,11.11,2.637727,1.33,4.62,25.0
316017,Other page 2005 Macquarie Fields riot 2005 Cro...,10.714000,6.456250,3.89,10.00,3.042500,1.43,4.90,25.0
7423,Saint-Denis be founded by Ã tienne RÃ gnault i...,8.887000,5.613637,3.69,9.39,2.545454,1.43,4.96,20.0
201052,Pougny be a commune in the Ain department in e...,7.722873,6.232500,2.89,12.05,2.505000,1.43,3.30,11.0
...,...,...,...,...,...,...,...,...,...
219270,"Jimmy Jimbo Donal Wales be bear on August 7 , ...",11.549000,4.587500,3.58,5.65,3.390000,1.85,4.88,10.0
215965,The M16 be an assault rifle use by the United ...,7.056838,5.545714,3.69,10.21,2.459375,1.38,4.85,21.0
241104,Discography and release,10.675467,6.550000,4.57,8.53,2.400000,1.52,3.28,3.0
194078,Prokofiev compose the score for Eisenstein ' s...,7.969984,6.947500,3.98,13.00,2.244286,1.43,4.71,26.0


In [288]:
parameters = {
    'clf__alpha': [0.001,0.01,0.1,1,5,10,50, 100]
}

grid_bnb_all_f1= (GridSearchCV(pipeline,
                                  param_grid=parameters,
                                  scoring='f1',
                                  n_jobs=-1, 
                                  cv=3)).fit(X_train,y_train)



/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_discretization.py:233: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 5 are removed. Consider decreasing the number of bins.
  "decreasing the number of bins." % jj


In [293]:

grid_bnb_all_f1.best_estimator_

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('mads',
                                                  Pipeline(steps=[('impute_mean',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler()),
                                                                  ('bin',
                                                                   KBinsDiscretizer(n_bins=4))]),
                                                  ['d_chall_score', 'aoa_mean',
                                                   'aoa_min', 'aoa_max',
                                                   'conc_rating_mean',
                                                   'conc_rating_min',
                                                   'conc_rating_max',
                                             

In [191]:
# best_model = grid_bnb_all_f1.best_estimator_.named_steps['clf']
# grid_bnb_all_f1.feature_names_in_





#transformerVectoriser.transformers_
# neg_class = best_model.feature_log_prob_[0, :].argsort()[::-1]
# pos_class = best_model.feature_log_prob_[1, :].argsort()[::-1]

# print(np.take(, neg_class[:10]))
# print(np.take(, pos_class[:10]))


# # Negative Class Feature Importance by Log Probabilities
# list(zip(np.take(cols, neg_class[:10]),
#          [best_model.feature_log_prob_[0,idx] for idx in neg_class]))


AttributeError: ignored

In [294]:
# Run model based on best parameters from gridsearch

model_name = 'Tf-idf + MADS BNB alpha .001'
# compute metrics from the test split data
compute_metrics(grid_bnb_all_f1.best_estimator_,model_name,X_train, y_train, X_test, y_test)

In [295]:
pd.DataFrame(metrics_df)

,model_name,model_instance,train_accuracy,train_precision,train_recall,train_aucprc,train_aucroc,test_accuracy,test_precision,test_recall,test_aucprc,test_aucroc
0,Tf-idf BNB alpha_.001 min_df_30 ngram_1_2,"(TfidfVectorizer(min_df=30, ngram_range=(1, 2)...",0.684894,0.718241,0.646273,"([0.5257042424035254, 0.5258272145244425, 0.52...",0.757069,0.665309,0.697354,0.627231,"([0.5280696332921302, 0.5280574798485745, 0.52...",0.734647
1,MADS BNB alpha .001,"(SimpleImputer(), MinMaxScaler(), BernoulliNB(...",0.520343,0.519756,0.994928,"([0.5189033099771687, 0.5197554278597107, 0.51...",0.501709,0.519852,0.519504,0.994977,"([0.518915909608523, 0.5195039975556348, 1.0, ...",0.501195
2,MADS BNB alpha .001,"(SimpleImputer(), StandardScaler(), BernoulliN...",0.612439,0.623923,0.637186,"([0.5189033099771687, 0.5209164530313577, 0.52...",0.651824,0.608574,0.620319,0.633327,"([0.518915909608523, 0.5208710047115752, 0.522...",0.649156
3,Tf-idf + MADS BNB alpha .001,"(ColumnTransformer(transformers=[('mads',\n ...",0.679437,0.699613,0.669828,"([0.5257222132013822, 0.5257208621288864, 0.52...",0.749218,0.664752,0.685115,0.654979,"([0.5288731849482887, 0.5288610337356855, 0.52...",0.732518


# Sentence Embeddings

In [296]:
from numpy import loadtxt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [297]:
embeddings = loadtxt('drive/Shareddrives/Milestone 2/embeddings.csv', delimiter=',')
embeddings

array([[-0.10521097,  0.04652381,  0.09932816, ..., -0.53146267,
         0.24081262,  0.04764764],
       [-0.09595645,  0.28408318,  0.0416585 , ..., -0.6049515 ,
         0.23389882,  0.2116979 ],
       [ 0.00624134,  0.04496894,  0.28006756, ..., -0.21483139,
         0.40460399, -0.12047322],
       ...,
       [-0.05062665,  0.11346165,  0.37695   , ..., -0.35318942,
         0.46983531, -0.04529605],
       [-0.13691955,  0.46627818, -0.0528408 , ..., -0.55561154,
         0.56975963,  0.25214309],
       [-0.23494626,  0.30814424,  0.28275521, ..., -0.56363898,
         0.44196188,  0.01724233]])

In [194]:
embeddings.shape

(395169, 100)

In [298]:
# Combine the embeddings and the original_text + Mads features
emb = pd.DataFrame(embeddings)
new_df = pd.concat([df,emb], axis=1)
new_df.head()

,original_text,lemmatized_text,d_chall_score,aoa_mean,aoa_min,aoa_max,conc_rating_mean,conc_rating_min,conc_rating_max,num_lemmas,...,90,91,92,93,94,95,96,97,98,99
0,There is manuscript evidence that Austen conti...,There be manuscript evidence that Austen conti...,8.533221,5.809310,3.57,12.12,2.495517,1.33,4.57,38.0,...,-0.000574,-0.282132,0.048765,0.281763,-0.562752,-0.018829,-0.266238,-0.531463,0.240813,0.047648
1,"In a remarkable comparative analysis , Mandaea...","In a remarkable comparative analysis , Mandaea...",12.320171,7.402308,2.89,11.94,2.334286,1.46,4.93,21.0,...,0.142222,-0.103547,-0.090528,0.095581,-0.276107,0.028346,-0.250517,-0.604951,0.233899,0.211698
2,"Before Persephone was released to Hermes , who...","Before Persephone be release to Hermes , who h...",5.931500,5.231351,2.78,11.17,2.556486,1.43,4.86,40.0,...,-0.042911,-0.127006,-0.027232,0.002538,-0.531988,-0.153132,-0.162387,-0.214831,0.404604,-0.120473
3,Cogeneration plants are commonly found in dist...,Cogeneration plant be commonly find in distric...,7.015012,6.742000,3.56,11.53,3.369655,1.52,4.93,32.0,...,0.113040,-0.480430,-0.121593,0.049611,-0.675361,0.358477,-0.145952,-0.454261,0.618850,0.002099
4,Geneva is the second-most-populous city in Swi...,Geneva be the second-most-populous city in Swi...,7.550745,5.455000,3.69,12.62,2.399333,1.43,4.79,22.0,...,-0.003146,-0.214928,-0.110112,0.408002,-0.512044,0.292142,-0.294712,-0.306517,0.631877,-0.067776


In [299]:
#Extract training
X_train_indexes = X_train.index
X_train_emb = new_df.iloc[X_train_indexes, 11:]



# Extract just the embedding data from test set
X_test_indexes = X_test.index
X_test_emb = new_df.iloc[X_test_indexes, 11:]

X_train_emb.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
91224,-0.372383,0.221127,0.418656,-0.075037,0.278601,0.282382,-0.179060,0.063454,-0.416448,0.017981,...,-0.318357,0.059500,-0.174866,0.281088,-0.411315,-0.156737,-0.283365,-0.481805,0.567332,0.058942
117728,0.016294,-0.113769,0.293257,-0.168054,0.222634,0.146599,0.070255,0.014296,-0.186127,0.057335,...,-0.083942,-0.180034,-0.202958,0.465693,-0.490053,0.123509,-0.455105,-0.099749,0.473420,-0.061285
316017,-0.073500,0.113797,0.029153,-0.429477,0.063771,0.347380,-0.124431,0.282229,-0.075735,0.068369,...,0.054707,-0.032472,-0.121287,0.100465,-0.510452,0.222725,-0.246233,-0.182647,0.495054,0.045244
7423,0.029224,-0.239776,0.132184,0.049248,0.145883,0.229183,-0.007984,0.014910,-0.091277,-0.008269,...,0.064875,-0.315753,-0.039297,0.541282,-0.457863,0.255201,-0.281852,-0.214106,0.434917,-0.177841
201052,-0.046389,-0.126177,0.413811,-0.052786,0.227497,0.208976,0.116180,0.549721,0.018257,0.237596,...,0.318063,-0.234182,-0.218740,0.573339,-0.420712,0.340783,-0.341281,-0.248862,0.539164,-0.111740


In [212]:
from sklearn.preprocessing import StandardScaler

In [305]:
embed_pipeline = Pipeline([('scaler', StandardScaler()),
                          ('clf',BernoulliNB())
                         ])

parameters = {
    'clf__alpha': [0.0001,0.001,0.01,0.1,1,5,10,50, 100,200,500]
}


grid_bnb_emb_f1 = (GridSearchCV(embed_pipeline,
                                 param_grid=parameters,
                                 scoring='f1',
                                 n_jobs=-1, 
                                 cv=3).fit(X_train_emb,y_train))


In [306]:
grid_bnb_emb_f1.best_params_

{'clf__alpha': 100}

In [308]:
grid_bnb_emb_f1.best_estimator_.score(X_train_emb,y_train)
# grid_mnb_emb_f1.best_estimator_.score(X_test_emb, y_test)

0.5782534612486363

(39517, 100)

In [309]:
# mnb = MultinomialNB(alpha=.001).fit(X_train_emb_mm, y_train)
model_name = 'embedding BNB alpha 100'
# X_test_emb_mm = scaler.transform(X_test_emb)
compute_metrics(grid_bnb_emb_f1.best_estimator_,
                model_name,
                X_train_emb,
                y_train,
                X_test_emb,
                y_test)

In [310]:
pd.DataFrame(metrics_df)

,model_name,model_instance,train_accuracy,train_precision,train_recall,train_aucprc,train_aucroc,test_accuracy,test_precision,test_recall,test_aucprc,test_aucroc
0,Tf-idf BNB alpha_.001 min_df_30 ngram_1_2,"(TfidfVectorizer(min_df=30, ngram_range=(1, 2)...",0.684894,0.718241,0.646273,"([0.5257042424035254, 0.5258272145244425, 0.52...",0.757069,0.665309,0.697354,0.627231,"([0.5280696332921302, 0.5280574798485745, 0.52...",0.734647
1,MADS BNB alpha .001,"(SimpleImputer(), MinMaxScaler(), BernoulliNB(...",0.520343,0.519756,0.994928,"([0.5189033099771687, 0.5197554278597107, 0.51...",0.501709,0.519852,0.519504,0.994977,"([0.518915909608523, 0.5195039975556348, 1.0, ...",0.501195
2,MADS BNB alpha .001,"(SimpleImputer(), StandardScaler(), BernoulliN...",0.612439,0.623923,0.637186,"([0.5189033099771687, 0.5209164530313577, 0.52...",0.651824,0.608574,0.620319,0.633327,"([0.518915909608523, 0.5208710047115752, 0.522...",0.649156
3,Tf-idf + MADS BNB alpha .001,"(ColumnTransformer(transformers=[('mads',\n ...",0.679437,0.699613,0.669828,"([0.5257222132013822, 0.5257208621288864, 0.52...",0.749218,0.664752,0.685115,0.654979,"([0.5288731849482887, 0.5288610337356855, 0.52...",0.732518
4,embedding BNB alpha 100,"(MinMaxScaler(), BernoulliNB(alpha=0.001))",0.519109,0.519010,0.999967,"([0.5190069210672111, 0.5190084875892209, 0.51...",0.500480,0.519144,0.519035,0.999951,"([0.5189684407663301, 0.5189694009263243, 0.51...",0.500480
5,embedding BNB alpha 100,"(StandardScaler(), BernoulliNB(alpha=100))",0.578253,0.587915,0.626051,"([0.5189062280331787, 0.518904875312457, 0.518...",0.602765,0.574867,0.584984,0.622013,"([0.5189553069798046, 0.5189431326398907, 0.51...",0.599505


# Sentence Embeddings + Mads Feaures

In [311]:
model_name = "emb + mads"

In [312]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler, Normalizer

In [319]:
X_train_emb_mads = new_df.iloc[X_train.index].copy()
X_test_emb_mads = new_df.iloc[X_test.index].copy()

X_train_emb_mads.drop(['original_text','lemmatized_text','label'],inplace=True,axis=1)
X_test_emb_mads.drop(['original_text','lemmatized_text','label'],inplace=True,axis=1)

# impute_columns = ['conc_rating_mean','conc_rating_min','conc_rating_max']
# mads_imputer = Pipeline([('imputer', SimpleImputer(strategy='mean'))
#                          ])

# preprocessor = ColumnTransformer(transformers=[('impute', mads_imputer,impute_columns)])


# embed_pipeline = Pipeline([('preprocessor',preprocessor),
#                            ('scaler', StandardScaler()),
#                            ('clf', BernoulliNB())
#                          ])
# param_grid = {
#     'clf__alpha': [0.001, 0.01, 0.1,1,2,3,4,5,10,50, 100,500]
# }

In [320]:

mads_pipe = Pipeline([
              ('impute_mean',SimpleImputer(strategy='mean')),
              ])

preprocessor = ColumnTransformer(transformers=[
    ('mads',mads_pipe,cols)
    ])


              
pipeline = Pipeline([('preprocessor', preprocessor),
                     ('scaler',StandardScaler()),
                     ('clf', BernoulliNB())  # classifier
                     ])


param_grid = {
    'clf__alpha': [0.001, 0.01, 0.1,1,2,3,4,5,10,50, 100,500]
}

In [321]:
grid_bnb_embmads_f1 = GridSearchCV(pipeline,
                                    param_grid=param_grid,
                                    scoring='f1',
                                    n_jobs=-1,
                                    cv=3)
grid_bnb_embmads_f1.fit(X_train_emb_mads,y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('mads',
                                                                         Pipeline(steps=[('impute_mean',
                                                                                          SimpleImputer())]),
                                                                         ['d_chall_score',
                                                                          'aoa_mean',
                                                                          'aoa_min',
                                                                          'aoa_max',
                                                                          'conc_rating_mean',
                                                                          'conc_rating_min',
                                                                          'conc_rating_max',
  

In [322]:
grid_bnb_embmads_f1.best_params_

{'clf__alpha': 500}

In [323]:
grid_bnb_embmads_f1.best_estimator_.score(X_train_emb_mads,y_train)

0.6124385635396399

'emb + mads clf__alpha_0.001'

In [325]:
model_name += ' alpha .001'
compute_metrics(grid_bnb_embmads_f1.best_estimator_,model_name,X_train_emb_mads, y_train, X_test_emb_mads, y_test)

In [ ]:
#Earlier Code (pre-pipeline)

# X_train_emb_mads = np.hstack((X_train_emb_mm,
#                               X_train_mads_features))

# param_grid = {'alpha': [0.001, 0.01,0.1,.5,1,2,3,4,5,10,25,50,100,500]}
# grid_mnb_embmads_acc = GridSearchCV(MultinomialNB(), param_grid=param_grid)
# grid_mnb_embmads_acc.fit(X_train_emb_mads,y_train)

# grid_mnb_embmads_acc.best_params_

# Confirm the number of nan values based on the conc_rating_mean and fill with 0 
# X_test_mads_features[X_test_mads_features['conc_rating_mean'].isna()==True]
# X_test_mads_features.fillna(0,inplace=True)

# model_name = 'embedding NB alpha .001'
# compute_metrics(grid_mnb_embmads_acc.best_estimator_,model_name,X_test_emb_mads, y_test)

GridSearchCV(estimator=MultinomialNB(),
             param_grid={'alpha': [0.001, 0.01, 0.1, 0.5, 1, 2, 3, 4, 5, 10, 25,
                                   50, 100, 500]})

In [ ]:
metrics = pd.DataFrame(metrics_df)

In [ ]:
metrics

,model_name,model_instance,accuracy,precision,recall,aucprc,aucroc
0,Tf-idf NB alpha 5,"(TfidfVectorizer(min_df=25, ngram_range=(1, 3)...",0.632285,0.621785,0.743831,"([0.5302270259088794, 0.5302148785974711, 0.53...",0.696735
1,MADS NB alpha 10,"(SimpleImputer(), MinMaxScaler(), MultinomialN...",0.525015,0.522492,0.983322,"([0.518915909608523, 0.5189037351958701, 0.518...",0.643516
2,embedding NB alpha .001,"(MinMaxScaler(), MultinomialNB(alpha=0.001))",0.518916,0.518916,1.000000,"([0.5189421738580285, 0.5189299994938503, 0.51...",0.596900


In [ ]:
import pickle

In [ ]:
best_model = metrics.iloc[2]['model_instance']

In [ ]:
f = open('drive/Shareddrives/Milestone 2/nb_model.txt','wb')
pickle.dump(best_model,f)